In [1]:
pip install flask pandas joblib scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, render_template, jsonify
import pandas as pd
import pickle
import datetime

# Load the trained model
with open("solar_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

# Initialize Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("mainpage.html")  # Serve the main page

@app.route("/eee")
def prediction_page():
    return render_template("eee .html")  # Serve the prediction form page

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Debug: Print received form data
        print("Received form data:", request.form)

        # Extract input data
        try:
            source_key = float(request.form["sourceKey"])
            ambient_temp = float(request.form["ambientTemp"])
            module_temp = float(request.form["moduleTemp"])
            irradiation = float(request.form["irradiation"])
            date_str = request.form["date"]
            time_str = request.form["time"]
        except KeyError as e:
            return jsonify({"error": f"Missing field: {str(e)}"}), 400
        except ValueError:
            return jsonify({"error": "Invalid data format. Please enter numbers correctly."}), 400

        # Process datetime
        try:
            dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
            year, month, day, hour, minute = dt.year, dt.month, dt.day, dt.hour, dt.minute
            minutes_pass = (hour * 60) + minute
        except ValueError:
            return jsonify({"error": "Invalid date or time format."}), 400

        # Create input dataframe
        input_data = pd.DataFrame([{
            "SOURCE_KEY": source_key,
            "AMBIENT_TEMPERATURE": ambient_temp,
            "MODULE_TEMPERATURE": module_temp,
            "IRRADIATION": irradiation,
            "YEAR": 2005,
            "MONTH": 5,
            "DAY": 15,
            "HOUR": hour,
            "MINUTES": minute,
            "MINUTES_PASS": minutes_pass
        }])

        # Predict
        prediction = model.predict(input_data)[0]

        # Recommendation
        if prediction > 600:
                recommendation = (
                    "<h3>✅ High Solar Output: Excellent energy generation.</h3>"
                    "<p><strong>Recommendation:</strong> Maximize energy storage (if a battery system is available). "
                    "Schedule high-energy tasks during this period (e.g., EV charging, industrial equipment).</p>"
                    "<p><strong>Tips:</strong></p>"
                    "<ul>"
                    "<li>Run high-power appliances now to take full advantage of available energy.</li>"
                    "<li>Store excess energy in batteries for later use or sell it back to the grid (if applicable).</li>"
                    "<li>Monitor panel performance to ensure consistent output.</li>"
                    "</ul>"
                )
        elif 300 <= prediction <= 600:
                recommendation = (
                    "<h3>🔆 Moderate Solar Output: Good generation with balanced efficiency.</h3>"
                    "<p><strong>Recommendation:</strong> Optimal energy output. Focus on efficient energy distribution and usage.</p>"
                    "<p><strong>Tips:</strong></p>"
                    "<ul>"
                    "<li>Run necessary appliances as needed.</li>"
                    "<li>Avoid unnecessary wastage; conserve energy where possible.</li>"
                    "<li>Consider partially charging backup batteries.</li>"
                    "</ul>"
            )
        elif 50 <= prediction < 300:
                    recommendation = (
                        "<h3>⚠️ Low Solar Output: Limited energy available.</h3>"
                        "<p><strong>Recommendation:</strong> Moderate output. Improving panel efficiency may help.</p>"
                        "<p><strong>Tips:</strong></p>"
                        "<ul>"
                        "<li>Use only essential appliances.</li>"
                        "<li>Check for shading or dirt on solar panels.</li>"
                        "<li>Evaluate panel tilt and orientation to improve output.</li>"
                        "<li>Consider expanding solar capacity or upgrading system components.</li>"
                        "</ul>"
                    )
        else:
            recommendation = (
                "<h3>🚨 Very Low Solar Output: Insufficient for peak energy needs.</h3>"
                "<p><strong>Recommendation:</strong> Switch to backup power sources such as the grid or a generator. "
                "Reduce usage of non-essential devices.</p>"
                "<p><strong>Tips:</strong></p>"
                "<ul>"
                "<li>Power only critical systems (lighting, communication, medical, etc.).</li>"
                "<li>Delay high-energy tasks until solar levels improve.</li>"
                "<li>If possible, schedule energy-heavy operations for high-sunlight hours.</li>"
                "<li>Explore adding battery storage to handle future downtimes.</li>"
                "</ul>"
    )


        return jsonify({
            "prediction": f"{prediction:.2f} kW",
            "recommendation": recommendation
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [7]:

%tb


IndentationError: unindent does not match any outer indentation level (<string>, line 66)